In [ ]:
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
path = "photo/p5.jpg"

In [ ]:
img = cv2.imread(path)
h, w, k = img.shape
if h > w:
    img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    h, w, k = img.shape

fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(img)

In [ ]:
def detect_edges(img):
    _img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, _img = cv2.threshold(_img, 191, 255, 0)
    return cv2.Canny(_img, 100, 200)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(detect_edges(img))

In [ ]:
def get_lines(img):
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 80  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 50  # minimum number of pixels making up a line
    max_line_gap = 4  # maximum gap in pixels between connectable line segments
    return cv2.HoughLinesP(
                img,
                rho,
                theta,
                threshold,
                np.array([]),
                min_line_length,
                max_line_gap
            )

In [ ]:
def get_max_length_line(img):
    max_length = 0
    rotation_angle = 0
    pt1 = (0, 0)
    pt2 = (0, 0)
    
    for line in get_lines(img):
        x1, y1, x2, y2 = line[0]
        length = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
        if length > max_length:
            max_length = length
            diff = 0
            if y1 > y2:
                diff = x1 - x2
            else:
                diff = x2 - x1
            rotation_angle = math.acos(abs(diff) / length) * 180 / math.pi
            if diff < 0:
                rotation_angle = -rotation_angle
            pt1 = (x1, y1)
            pt2 = (x2, y2)
    
    return pt1, pt2, max_length, rotation_angle
    

In [ ]:
def draw_line(img, pt1, pt2):
    _img = img.copy()
    cv2.line(_img, pt1, pt2, (255, 0, 0), 2)
    fig, ax = plt.subplots(figsize=(18, 12))
    ax.imshow(_img)

In [ ]:
pt1, pt2, max_length, rotation_angle = get_max_length_line(detect_edges(img))
draw_line(img, pt1, pt2)

print(f"Max length line: {max_length}")
print(f"Rotation angle: {rotation_angle}")

In [ ]:
M = cv2.getRotationMatrix2D((w // 2, h // 2), rotation_angle, 1)
img = cv2.warpAffine(img, M, (w, h))

fig, ax = plt.subplots(figsize=(18, 12))
ax.imshow(img)